In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/image_classification_flask/

/content/drive/My Drive/image_classification_flask


In [3]:
import tensorflow as tf
import numpy as np
import urllib.request
from keras.models import load_model
from keras.preprocessing import image
from keras.applications import imagenet_utils

In [4]:
!pip install flask_ngrok

In [5]:
def save_and_get_img_path(url):
  url_path = url
  file_name = url.split('/')[-1]
  urllib.request.urlretrieve(url_path, filename=file_name)
  return file_name

In [6]:
def prepare_image(img_path):
  img = image.load_img(img_path, target_size=(img_width, img_height))
  img_array = image.img_to_array(img)
  img_array_expand_dims = np.expand_dims(img_array, axis=0)
  return tf.keras.applications.mobilenet.preprocess_input(img_array_expand_dims)

In [7]:
from flask import Flask, request, Response
from flask_ngrok import run_with_ngrok
import json

In [8]:
app = Flask(__name__)
img_width, img_height = 128, 128

model = load_model('best_model.h5')

class_names = ['바베큐 폭림',
 '비빔밥',
 '샌드위치',
 '구운 연어',
 '핫도그',
 '스테이크']

@app.route('/')
def hello_world():
  return "just started"

@app.route('/predict', methods=['GET', 'POST'])
def predict_image():
  url = request.args['url']
  img_path = save_and_get_img_path(url)
  preprocess_image = prepare_image(img_path)
  predictions = model.predict(preprocess_image)
  return json.dumps(class_names[np.argmax(predictions)], ensure_ascii=False)

In [ ]:
run_with_ngrok(app)
app.run()